<a href="https://colab.research.google.com/github/wesleyscholl/boilerplate-neural-network-sms-text-classifier/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam".
# A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

# You should create a function called predict_message that takes a message string as an argument and returns a list.
# The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1).
# The second element in the list should be the word "ham" or "spam", depending on which is most likely.

# For this challenge, you will use the SMS Spam Collection dataset. The dataset has already been grouped into train data and test data.

# The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  """Predicts whether a message is ham or spam using a trained machine learning model.

  Args:
    pred_text: The message string to be predicted.

  Returns:
    A list containing the prediction value (between 0 and 1) and the predicted label ('ham' or 'spam').
  """

  # Load training and validation data from TSV files using pandas.
  train_data = pd.read_csv('train-data.tsv', sep='\t', header=None, names=['label', 'message'])
  test_data = pd.read_csv('valid-data.tsv', sep='\t', header=None, names=['label', 'message'])

  # Convert labels to numerical values: 'ham' -> 0, 'spam' -> 1
  train_data['label'] = train_data['label'].map({'ham': 0, 'spam': 1})
  test_data['label'] = test_data['label'].map({'ham': 0, 'spam': 1})

  # Create a TensorFlow tokenizer to convert text to numerical sequences.
  # num_words: Maximum number of words to keep in the vocabulary.
  # oov_token: Token for out-of-vocabulary words.
  tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=10000, oov_token='<OOV>')

  # Fit the tokenizer on the training messages to build the vocabulary.
  tokenizer.fit_on_texts(train_data['message'])

  # Convert training and validation messages to numerical sequences using the tokenizer.
  train_sequences = tokenizer.texts_to_sequences(train_data['message'])
  test_sequences = tokenizer.texts_to_sequences(test_data['message'])

  # Pad sequences to ensure uniform length for the model input.
  # padding='post': Add padding to the end of sequences.
  # maxlen=120: Maximum length of the sequences.
  train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, padding='post', maxlen=120)
  test_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, padding='post', maxlen=120)

  # Create a sequential model using Keras.
  model = tf.keras.Sequential([
      # Embedding layer to represent words as dense vectors.
      tf.keras.layers.Embedding(10000, 16),
      # Global average pooling to reduce the dimensionality of the embeddings.
      tf.keras.layers.GlobalAveragePooling1D(),
      # Dense layer with ReLU activation for feature extraction.
      tf.keras.layers.Dense(16, activation='relu'),
      # Output layer with sigmoid activation for binary classification (ham/spam).
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  # Compile the model with appropriate loss function, optimizer, and metrics.
  # loss='binary_crossentropy': Suitable for binary classification.
  # optimizer='adam': A common optimization algorithm.
  # metrics=['accuracy']: To evaluate the model's performance.
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

  # Train the model using the training data and validate with validation data.
  # epochs=30: Increased number of times the model iterates over the entire training data.
  # validation_data: Data used to monitor the model's performance during training.
  model.fit(train_padded, train_data['label'], epochs=60, validation_data=(test_padded, test_data['label']))

  # Preprocess the input message for prediction.
  pred_sequence = tokenizer.texts_to_sequences([pred_text])
  pred_padded = tf.keras.preprocessing.sequence.pad_sequences(pred_sequence, padding='post', maxlen=120)

  # Make the prediction using the trained model.
  prediction_value = model.predict(pred_padded)[0][0]

  # Determine the predicted label based on the prediction value.
  prediction_label = 'ham' if prediction_value < 0.5 else 'spam'

  # Return the prediction value and label.
  return [prediction_value, prediction_label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
